In [45]:
import pandas as pd

df = pd.read_csv('hasil_kategori_lda.csv')
df.head()

,Unnamed: 0,id,review,label,pred_single,pred_multiple,terms
0,0,0,i am not necessarily fanatical about this plac...,VALUE,VALUE,VALUE|VALUE,place|price
1,1,2,the high prices you're going to pay is for the...,VALUE,VALUE,VALUE|VALUE,price|going
2,2,3,the high prices you're going to pay is for the...,VALUE,VALUE,VALUE|VALUE,price|going
3,3,4,the high prices you're going to pay is for the...,VALUE,VALUE,VALUE|VALUE,price|going
4,4,5,the high prices you're going to pay is for the...,VALUE,VALUE,VALUE|VALUE,price|going


In [46]:
def calculate(label, is_multi):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    kosong = 0
    
    for index in range(0, len(df)):
        labels = df['label'][index].split('|')
        if is_multi:
            if type(df['pred_multiple'][index]) != float:
                preds = df['pred_multiple'][index].split('|')

                if label in labels and label in preds:
                    tp += 1
                elif label not in labels and label not in preds:
                    tn += 1
                elif label not in labels and label in preds:
                    fp += 1
                elif label in labels and label not in preds:
                    fn += 1
            else:
                fn += 1
                kosong += 1
        else:
            if type(df['pred_single'][index]) != float:
                preds = df['pred_single'][index]

                if label in labels and label in preds:
                    tp += 1
                elif label not in labels and label not in preds:
                    tn += 1
                elif label not in labels and label in preds:
                    fp += 1
                elif label in labels and label not in preds:
                    fn += 1
            else:
                fn += 1
                kosong += 1
                
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * ((precision * recall) / (precision + recall))
    print('kosong: ', kosong)
    return precision, recall, f1

In [48]:
cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'VALUE']
for c in cat:
    print(calculate(c, False))

kosong:  79
(0.7483443708609272, 0.40942028985507245, 0.5292740046838408)
kosong:  79
(0.956989247311828, 0.413953488372093, 0.577922077922078)
kosong:  79
(0.9818181818181818, 0.22131147540983606, 0.3612040133779264)
kosong:  79
(0.42543859649122806, 0.5187165775401069, 0.4674698795180723)


In [44]:
cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'VALUE']
for c in cat:
    print(calculate(c, True))

kosong:  79
(0.6572769953051644, 0.5072463768115942, 0.5725971370143149)
kosong:  79
(0.9467213114754098, 0.5372093023255814, 0.6854599406528189)
kosong:  79
(0.9307692307692308, 0.4959016393442623, 0.6470588235294118)
kosong:  79
(0.32628398791540786, 0.5775401069518716, 0.41698841698841693)
